In [12]:
# Importing library
import pandas as pd
import os
import glob
import warnings
warnings.filterwarnings('ignore')
import math 
import mysql.connector as msql

In [13]:
# Displaying all columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [14]:
# Taking files from define path which are in csv format
path = r'C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives-LAPTOP-D2TFS89H\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\New_Assignment_Scrapping Method\Source File'

# Change the current working directory to the specified path
os.chdir(path)

# Use glob to get a list of file paths for all CSV files in the current directory
file_paths = glob.glob('*.csv')

In [15]:
# Create an empty list to store the keywords
keywords = []

# Create an empty dictionary to store the dataframes
dataframes = {}

# Iterate through each file path in the list of file paths
for file in file_paths:
    # Find the starting position of the keyword by searching for 'Assignment_'
    start = file.find('Assignment_') + len('Assignment_')
    
    # Find the ending position of the keyword by searching for '.csv'
    end = file.rfind('.csv')
    
    # Extract the keyword from the file path, removing leading/trailing spaces, and replacing spaces with underscores
    keyword = file[start:end].strip().replace(' ', '_')
    
    # Add the extracted keyword to the list of keywords
    keywords.append(keyword)
    
    # Read the CSV file and store it in a dataframe with the keyword as the key in the dataframes dictionary
    dataframes[keyword] = pd.read_csv(file)


In [16]:
# List of keyword of our csv file present in our folder
keywords

['Career_Action_Plan_(CAP)',
 'Critical_thinking',
 'Masteclass_1_-_Career_Exploration',
 'Masterclass_-_Academic_CV_and_Industry_Resume',
 'Networking__Build_a_World-Class_LinkedIn_Profile',
 'Planning_&_Applying_for_Masters_in_India_&_Abroad',
 'RIASEC_personality_test',
 'sir',
 'SMART_goal',
 'SWOT',
 'mam333']

In [17]:
# Iterate through each keyword in the list of keywords
for keyword in keywords:
    # Assign the corresponding dataframe to a variable with the same name as the keyword
    # This is done by using the globals() function to create a new variable in the global namespace
    globals()[keyword] = dataframes[keyword]

    # We have use golabl function to create dataframe of keyword so it should take name of dataframe as keyword. Like Swot will be name of datafarme.

In [18]:
# Reading Kalpana SHE for STEM Incubator file from Source files

directory_path =(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives-LAPTOP-D2TFS89H\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Incubator New Remapping Code\Kalpana Source File\02_Source_Kalpana incubator")

csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    Kalpana = pd.read_csv(file_path,usecols=['Email'])
    print(f"Data from {file}:")

Data from Kalpana Incubator Trial.csv:


In [46]:
# Create an empty dataframe to store the final results
final_df = pd.DataFrame()

# Create a dictionary to store comments for each keyword
comments_dict = {}

# Iterate through each keyword in the list of keywords
for keyword in keywords:
    # Filter the columns based on specific conditions...
    filtered_columns = [col for col in globals()[keyword] if ('message' in col) or ('email' in col) or (col == 'status')]

    new_df = globals()[keyword][filtered_columns]
    
    message_columns = [col for col in new_df.columns if 'message' in col]
    new_df[message_columns] = new_df[message_columns].fillna('')
    
    # Create a new 'Comment' column by joining the values of the 'message' columns for each row
    comment_col_name = 'Comment_' + keyword
    new_df[comment_col_name] = new_df[message_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    
    # Store the comments column name in the dictionary
    comments_dict[keyword] = comment_col_name
    
    # Modify the 'Email' column by removing a specific prefix from the values
    new_df['Email'] = new_df['user/email'].str.replace('vigyanshaalainternational1617-', '')
    
    # Rename the 'status' column to the keyword
    new_df = new_df.rename(columns={'status': keyword})
    
    # Select the desired columns for the new dataframe
    selected_columns = ['Email', keyword, comment_col_name]
    new_df = new_df[selected_columns]
    
    # If the final dataframe is empty, assign it as the new_df
    if final_df.empty:
        final_df = new_df.copy()
    # If the final dataframe is not empty, merge the new_df with the existing final dataframe
    else:
        final_df = pd.merge(final_df, new_df, on='Email', how='left')



In [48]:
# Finding columns name of dataframe
final_df.columns

Index(['Email', 'Career_Action_Plan_(CAP)', 'Comment_Career_Action_Plan_(CAP)',
       'Critical_thinking', 'Comment_Critical_thinking',
       'Masteclass_1_-_Career_Exploration',
       'Comment_Masteclass_1_-_Career_Exploration',
       'Masterclass_-_Academic_CV_and_Industry_Resume',
       'Comment_Masterclass_-_Academic_CV_and_Industry_Resume',
       'Networking__Build_a_World-Class_LinkedIn_Profile',
       'Comment_Networking__Build_a_World-Class_LinkedIn_Profile',
       'Planning_&_Applying_for_Masters_in_India_&_Abroad',
       'Comment_Planning_&_Applying_for_Masters_in_India_&_Abroad',
       'RIASEC_personality_test', 'Comment_RIASEC_personality_test', 'sir',
       'Comment_sir', 'SMART_goal', 'Comment_SMART_goal', 'SWOT',
       'Comment_SWOT', 'mam333', 'Comment_mam333'],
      dtype='object')

In [49]:
# Create a list of columns to assign, excluding 'Email' and columns starting with 'Comment'
assign = [col for col in final_df.columns if col != 'Email' and not col.startswith('Comment')]


In [50]:
# Remove leading/trailing spaces in the selected columns
final_df[assign] = final_df[assign].applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [51]:
# To get unique values of dataframe to see what columns it consist to give 'marks'
final_df['Masterclass_-_Academic_CV_and_Industry_Resume'].unique()

array([nan, 'rejected', 'reviewed', 'under review'], dtype=object)

In [52]:
# To get unique values of dataframe to see what columns it consist to give 'marks'
final_df['SWOT'].unique()

array(['rejected', 'reviewed', nan, 'under review'], dtype=object)

### If you get new unique values in our column then add marks to it. such example "Pass" then at Please add pass in below dict and give marks to it like replace_dict = {'under review': 30, 'reviewed': 30,'rejected': 80, 'Pass': 100}

# Below you can give marks for Assignment 👇⬇️

In [53]:
replace_dict = {'under review': 30, 'reviewed': 30,'rejected': 80}


# 👆☝️


In [54]:
# Replace values in selected columns
final_df[assign] = final_df[assign].replace(replace_dict)

In [55]:
# Convert the selected columns to float
final_df[assign] = final_df[assign].astype(float)

In [56]:
# Filling NaN Values with zero
final_df[assign]=final_df[assign].fillna(0)

In [57]:
# Calculate the sum of selected columns
sum_of_scores = final_df[assign].sum(axis=1)

# Count the number of selected columns
num_of_columns = len(assign)

# Calculate the average score
final_df['Assignment_Score'] = sum_of_scores / num_of_columns


In [58]:
# Rounding Off values of Assignment score to two digits
final_df['Assignment_Score']=final_df['Assignment_Score'].round(2)

In [59]:
# Filling NaN Values with empty
final_df=final_df.fillna('')

In [60]:
# Checking our data
final_df.head()

,Email,Career_Action_Plan_(CAP),Comment_Career_Action_Plan_(CAP),Critical_thinking,Comment_Critical_thinking,Masteclass_1_-_Career_Exploration,Comment_Masteclass_1_-_Career_Exploration,Masterclass_-_Academic_CV_and_Industry_Resume,Comment_Masterclass_-_Academic_CV_and_Industry_Resume,Networking__Build_a_World-Class_LinkedIn_Profile,Comment_Networking__Build_a_World-Class_LinkedIn_Profile,Planning_&_Applying_for_Masters_in_India_&_Abroad,Comment_Planning_&_Applying_for_Masters_in_India_&_Abroad,RIASEC_personality_test,Comment_RIASEC_personality_test,sir,Comment_sir,SMART_goal,Comment_SMART_goal,SWOT,Comment_SWOT,mam333,Comment_mam333,Assignment_Score
0,lankasrilekha2003@gmail.com,80.0,Successfully Submitted Wrong assignment submit...,80.0,Successfully Submitted Please elaborate more o...,80.0,Successfully Submitted Please go to this web p...,0.0,,0.0,,0.0,,80.0,Successfully Submitted Please take the RIASEC ...,80.0,Successfully Submitted Please watch the SWOT v...,80.0,Successfully Submitted please submit your smar...,80.0,Successfully Submitted Please watch the SWOT v...,80.0,Successfully Submitted Please go to this web p...,58.18
1,keerthika11aug@gmail.com,80.0,Successfully Submitted Wrong assignment submit...,80.0,Successfully Submitted Please elaborate more o...,0.0,,0.0,,0.0,,30.0,Successfully Submitted Approved!!!,30.0,Successfully Submitted Keep up the good work!,30.0,Successfully Submitted Good job. Now think abo...,30.0,Successfully Submitted well done,30.0,Successfully Submitted Good job. Now think abo...,0.0,,28.18
2,jaswanthgoud007@gmail.com,80.0,Successfully Submitted Wrong assignment submit...,0.0,,0.0,,0.0,,0.0,,0.0,,0.0,,0.0,,80.0,Successfully Submitted The Measurable is not r...,0.0,,0.0,,14.55
3,sravanthibichala4@gmail.com,80.0,Successfully Submitted Wrong assignment submit...,80.0,Successfully Submitted Please elaborate more o...,0.0,,80.0,Successfully Submitted Please take care of the...,0.0,,0.0,,30.0,Successfully Submitted Good job!,80.0,Successfully Submitted Please watch the SWOT v...,80.0,Successfully Submitted You may have to see all...,80.0,Successfully Submitted Please watch the SWOT v...,0.0,,46.36
4,jadalamounika233@gmail.com,80.0,Successfully Submitted DO NOT COPY. Re-watch t...,30.0,Successfully Submitted Approved !! Elaborate m...,30.0,Successfully Submitted Please watch this video...,30.0,Successfully Submitted Well Done! Please take ...,0.0,,0.0,,30.0,Successfully Submitted Good job!,30.0,Successfully Submitted Please watch the SWOT v...,30.0,Successfully Submitted Nice! Please work on me...,30.0,Successfully Submitted Please watch the SWOT v...,30.0,Successfully Submitted Please watch this video...,29.09


In [61]:
# Iterate through all columns in the dataframe
for col in final_df.columns:
    # Check if column name contains '.'
    if '.' in col:
        # Replace '.' with '_' and remove ' ' only where '.' is present in the column name
        new_col = col.replace('.', '_').replace(' ', '') if '.' in col else col
        # Rename the column
        df.rename(columns={col: new_col}, inplace=True)


In [63]:
# Cheaking the columns present in the dataframe
final_df.columns

Index(['Email', 'Career_Action_Plan_(CAP)', 'Comment_Career_Action_Plan_(CAP)',
       'Critical_thinking', 'Comment_Critical_thinking',
       'Masteclass_1_-_Career_Exploration',
       'Comment_Masteclass_1_-_Career_Exploration',
       'Masterclass_-_Academic_CV_and_Industry_Resume',
       'Comment_Masterclass_-_Academic_CV_and_Industry_Resume',
       'Networking__Build_a_World-Class_LinkedIn_Profile',
       'Comment_Networking__Build_a_World-Class_LinkedIn_Profile',
       'Planning_&_Applying_for_Masters_in_India_&_Abroad',
       'Comment_Planning_&_Applying_for_Masters_in_India_&_Abroad',
       'RIASEC_personality_test', 'Comment_RIASEC_personality_test', 'sir',
       'Comment_sir', 'SMART_goal', 'Comment_SMART_goal', 'SWOT',
       'Comment_SWOT', 'mam333', 'Comment_mam333', 'Assignment_Score'],
      dtype='object')

### The step remaing here is renaming i.e while putting to our mysql

In [66]:
# Get the list of column names in the final_df
column_names = final_df.columns

# Iterate through the column names and rename columns starting with "Comment"
for col in column_names:
    if col.startswith("Comment"):
        new_col_name = "Comment"
        final_df = final_df.rename(columns={col: new_col_name})



In [67]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Career_Action_Plan_(CAP)', 'Comment', 'Critical_thinking',
       'Comment', 'Masteclass_1_-_Career_Exploration', 'Comment',
       'Masterclass_-_Academic_CV_and_Industry_Resume', 'Comment',
       'Networking__Build_a_World-Class_LinkedIn_Profile', 'Comment',
       'Planning_&_Applying_for_Masters_in_India_&_Abroad', 'Comment',
       'RIASEC_personality_test', 'Comment', 'sir', 'Comment', 'SMART_goal',
       'Comment', 'SWOT', 'Comment', 'mam333', 'Comment', 'Assignment_Score'],
      dtype='object')

In [68]:
# Renaming the column to put in datafarme 
new_names = {
    'SWOT': 'WK_1_SWOT_Analysis',
    'Career_Action_Plan_(CAP)': 'WK_3_Career_Action_Plan',
    'Critical_thinking': 'WK_6_Critical_thinking',
    'Masteclass_1_-_Career_Exploration': 'MC_Career_Exploration',
    'Masterclass_-_Academic_CV_and_Industry_Resume': 'MC_CV_Resume',
    'Networking__Build_a_World-Class_LinkedIn_Profile': 'MC_LinkedIn_Profile',
    'Planning_&_Applying_for_Masters_in_India_&_Abroad': 'MC_Planning_Masters',
    'RIASEC_personality_test': 'WK_2_RIASEC',
    'SMART_goal': 'WK_2_SMART_goal'
}

for col in final_df.columns:
    if col in new_names:
        final_df.rename(columns={col: new_names[col]}, inplace=True)


In [69]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'WK_3_Career_Action_Plan', 'Comment', 'WK_6_Critical_thinking',
       'Comment', 'MC_Career_Exploration', 'Comment', 'MC_CV_Resume',
       'Comment', 'MC_LinkedIn_Profile', 'Comment', 'MC_Planning_Masters',
       'Comment', 'WK_2_RIASEC', 'Comment', 'sir', 'Comment',
       'WK_2_SMART_goal', 'Comment', 'WK_1_SWOT_Analysis', 'Comment', 'mam333',
       'Comment', 'Assignment_Score'],
      dtype='object')

In [70]:
# Renaming the 'comment' in our given format
cols = final_df.columns.tolist()
new_cols = []
prev_col = ''

for col in cols:
    if col == 'Comment':
        new_cols.append(f'Comments_{prev_col}')
    else:
        new_cols.append(col)
    prev_col = col

final_df.columns = new_cols


In [71]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'WK_3_Career_Action_Plan', 'Comments_WK_3_Career_Action_Plan',
       'WK_6_Critical_thinking', 'Comments_WK_6_Critical_thinking',
       'MC_Career_Exploration', 'Comments_MC_Career_Exploration',
       'MC_CV_Resume', 'Comments_MC_CV_Resume', 'MC_LinkedIn_Profile',
       'Comments_MC_LinkedIn_Profile', 'MC_Planning_Masters',
       'Comments_MC_Planning_Masters', 'WK_2_RIASEC', 'Comments_WK_2_RIASEC',
       'sir', 'Comments_sir', 'WK_2_SMART_goal', 'Comments_WK_2_SMART_goal',
       'WK_1_SWOT_Analysis', 'Comments_WK_1_SWOT_Analysis', 'mam333',
       'Comments_mam333', 'Assignment_Score'],
      dtype='object')

In [74]:
# Extracting desire output to csv sheet
final_df.to_csv(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives-LAPTOP-D2TFS89H\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\New_Assignment_Scrapping Method\Output\Webscraping_Assignment.csv",mode='w',index=False)

# Storing data on MySQL

In [29]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="kalpana",auth_plugin='mysql_native_password')
#cursor =conn.cursor()

☝️ REMOVE COMMENT OF ABOVE I.E REMOVE # to put data in database

In [30]:

# Get the existing columns in the database
cursor.execute("SHOW COLUMNS FROM kalpana.11_incubator_assignment_monitoring")
existing_columns = [col[0] for col in cursor.fetchall()]

# Define the column name before which the new column should be added
target_column = 'Assignment_Score'

# Check if any new columns exist in the dataframe but not in the database
new_columns = [col for col in final_df.columns if col not in existing_columns]
if new_columns:
    # Add new columns to the database before the target column
    for col in reversed(new_columns):
        if col not in existing_columns:
            # Get the index of the target column
            target_column_index = existing_columns.index(target_column)
            # Set the data type based on whether the column name starts with Comment
            data_type = "INT" if not col.startswith("Comment") else "TEXT"
            alter_query = f"ALTER TABLE kalpana.11_incubator_assignment_monitoring ADD COLUMN {col} {data_type} AFTER {existing_columns[target_column_index - 1]}"
            cursor.execute(alter_query)
            existing_columns.insert(target_column_index - 1, col)


In [31]:
# Your existing code for inserting data into the database table
for i, row in final_df.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(final_df.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO kalpana.11_incubator_assignment_monitoring ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in final_df.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))


In [32]:
# Commit the changes to the database
conn.commit()